# **41079 Computer Science Studio 2**
## *Notebook 3: Inference, Model Packaging, & Deployment*

---
## **1. Introduction & Set Up**

#### **1.1. Import Necessary Libaries/Packages**

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

#### **1.2. Load Trained Sentiment Model and Tokenizer**

In [3]:
MODEL_DIR = "./sentiment_model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
model.eval()

model.to(device)
print(f"Using device: {device}")

Using device: cuda


#### **1.3. Declare Label → Sentiment Mapping**

In [4]:
# Mapping for huamn-readable sentiment labels
label_to_sentiment = {
    0: "negative",
    1: "neutral",
    2: "positive"
}

---
## **2. Inference Functionality**

#### **2.1. Define Prediction Function**

In [61]:
def predict_sentiment(text: str, max_length: int = 64) -> str:
    """
    Predict sentiment label for a given input text.
    
    Args:
        text (str): The input sentence or phrase.
        max_length (int): Max token length for padding/truncation.
    
    Returns:
        str: Human-readable sentiment label (negative, neutral, positive).
    """
    encoded = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=max_length)
    encoded = {k: v.to(device) for k, v in encoded.items()}
    with torch.no_grad():
        outputs = model(**encoded)
        pred_label = torch.argmax(outputs.logits, dim=1).item()
    return label_to_sentiment.get(pred_label, "unknown")

#### **2.2. Define Prediction Function**

In [7]:
#### **2.2. Run Example Predictions**
sample_texts = [
    "I love this product!",
    "This is fine, nothing special.",
    "Absolutely terrible experience, would not recommend."
]

for text in sample_texts:
    sentiment = predict_sentiment(text)
    print(f"Text: {text}\n → Predicted Sentiment: {sentiment}\n")


Text: I love this product!
 → Predicted Sentiment: positive

Text: This is fine, nothing special.
 → Predicted Sentiment: neutral

Text: Absolutely terrible experience, would not recommend.
 → Predicted Sentiment: negative



---
## **3. Integration Notes**

#### **3.1. Expected Input**
- Raw strings representing messages, sentences, or chat input.
- Emojis and informal syntax are valid, handled via the `vinai/bertweet-large` tokenizer.

#### **3.2. Expected Output**
One of three sentiment classes:
1. `negative`,

2. `neutral`,

3. or `positive`

#### **3.3. Intended Deployment Environments**
This model can be easily wrapped into: 
- A Flask API

- Discord/Slack moderation bots

- Background services for chat systems

- Other such chat-based environments that could benefit from sentiment-analysis based moderation

#### **3.4. GPU/CPU Compatibility**
The functionality of this project is relatively hardware-flexible; the script uses GPU if available but automatically falls back to CPU.

---
## **4. Appendix**

#### **4.1. Batch Prediction Helper (Optional Functionality)**

In [8]:
def predict_batch(text_list, max_length=64):
    encoded = tokenizer(text_list, return_tensors="pt", truncation=True, padding=True, max_length=max_length)
    encoded = {k: v.to(device) for k, v in encoded.items()}
    with torch.no_grad():
        logits = model(**encoded).logits
        preds = torch.argmax(logits, dim=1).cpu().tolist()
    return [label_to_sentiment[p] for p in preds]

#### **4.2. Behaviour-Aware Moderation Demo (Customised Implementation Demo)**

This appendix includes a lightweight behavioural tracking demo that adapts the model's thresholding logic based on each user’s recent sentiment history. 

It is **not part of the core model**, but demonstrates how this package **could be integrated** into intelligent moderation pipelines.

In [38]:
import time

# Store user history and threshold evolution
user_history = {}
threshold_evolution = []

def predict_with_user_context(
    text,
    username,
    base_thresholds={0: 0.0, 1: 0.4, 2: 0.3},
    max_length=64,
    strike_policy="ramp",  # or "strict"
    strike_limit=3,
    timeout_seconds=60
):
    thresholds = base_thresholds.copy()
    current_time = time.time()
    
    history = user_history.get(username, {
        "last_sentiment": None,
        "strikes": 0,
        "timeout_until": 0
    })

    # If user is in timeout, ignore the message
    if current_time < history["timeout_until"]:
        remaining = int(history["timeout_until"] - current_time)
        return f"[SERVER] User '{username}' is muted for another {remaining} seconds."


    # Threshold ramping (if in ramp mode)
    if strike_policy == "ramp" and history["last_sentiment"] == "neutral":
        thresholds[1] += 0.05
        thresholds[2] += 0.05

    # Tokenise + move to device
    encoded = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=max_length)
    encoded = {k: v.to(model.device) for k, v in encoded.items()}

    # Run prediction
    with torch.no_grad():
        logits = model(**encoded).logits
        probs = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0]

    # Threshold-based decision
    if strike_policy == "strict" and history["strikes"] >= strike_limit:
        label = 0
    elif probs[1] > thresholds[1]:
        label = 1
    elif probs[2] > thresholds[2]:
        label = 2
    else:
        label = 0

    sentiment = label_to_sentiment[label]

    # Update history and handle punishment
    if sentiment == "neutral":
        history["strikes"] += 1
    elif sentiment == "negative":
        history["strikes"] = 0
        history["timeout_until"] = current_time + timeout_seconds
        print(f"[SERVER] Prevented User '{username}' from saying: \"{text}\"")
        return f"[SERVER] User '{username}' timed out for {timeout_seconds} seconds due to negative behaviour."
    else:
        history["strikes"] = 0

    history["last_sentiment"] = sentiment
    user_history[username] = history

    # Log thresholds
    threshold_evolution.append({
        "message": text,
        "user": username,
        "sentiment": sentiment,
        "strike_count": history["strikes"],
        "threshold_1": thresholds[1],
        "threshold_2": thresholds[2],
        "policy": strike_policy
    })

    return f"[CHAT] @{username}: \"{text}\" → {sentiment}"


In [ ]:
# Demo messages from a repeat user
user = "Adam"

messages = [
    "You sure are great!",
    "You're pretty good.",
    "You're meh I suppose...",
    "...Okay I take it back, you're dumb",
    "NO TAKE BACKSIES!"
]

for msg in messages:
    print(predict_with_user_context(msg, user, strike_policy="strict"))


[CHAT] @Adam: "You sure are great!" → positive
[CHAT] @Adam: "You're pretty good." → positive
[CHAT] @Adam: "You're meh I suppose..." → neutral
[SERVER] Prevented User 'Adam' from saying: "...Okay I take it back, you're dumb"
[SERVER] User 'Adam' timed out for 60 seconds due to negative behaviour.
[SERVER] User 'Adam' is muted for another 59 seconds.


#### **4.3. Behaviour With Nuanced Text**

In [123]:
def predict_messages(messages, title):
    print(f"===== {title} =====\n")
    for m in messages:
        sentiment = predict_sentiment(m)
        print(f"{m}\n  → {sentiment}\n")

In [ ]:
# Sarcasm/Phrases with Negative Words
messages = [
    "Oh great, another amazing update that breaks everything.",
    "This isn't bad at all.",
    "Well it's not the worst service I've used.",
    "Honestly, I think the update is fine, people are just whining."
]

predict_messages(messages, "Sarcasm/Phrases with Negative Words")

===== Sarcasm/Phrases with Negative Words =====

Oh great, another amazing update that breaks everything.
  → negative

This isn't bad at all.
  → positive

Well it's not the worst service I've used.
  → neutral

Honestly, I think the update is fine, people are just whining.
  → neutral



In [ ]:
# Classic Toxicity
messages = [
    "You guys are cringe as hell.",
    "You're hopeless. Just uninstall."
]

predict_messages(messages, "Classic Toxicity")

===== Classic Toxicity =====

You guys are cringe as hell.
  → negative

You're hopeless. Just uninstall.
  → negative



In [ ]:
# Code Switching
messages = [
    "That service was malísimo.", # "That service was awful."
    "c'était magnifique, wow..."  # "It was magnificent, wow..."
]

predict_messages(messages, "Code Switching")

===== Code Switching =====

That service was malísimo.
  → negative

c'était magnifique, wow...
  → positive



In [ ]:
# Factual Statements
messages = [
    "The file was uploaded at 4:03PM.",
    "We will meet in Room 204.",
]

predict_messages(messages, "Factual Statements")

===== Factual Statements =====

The file was uploaded at 4:03PM.
  → neutral

We will meet in Room 204.
  → neutral

